In [1]:
import pandas as pd
import numpy as np
import matplotlib as mplot
import seaborn as sb
import time

In [2]:
movies_genres_data=pd.read_csv("Movies_genres.csv")

In [3]:
def numeric_eliminator(total_text, index, column):
        string = ""
        #print(total_text.split())
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalpha()))
            # Conver all letters to lower-case
            word = word.lower()
            string += word + " "
        movies_genres_data[column][index] = string

In [4]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in movies_genres_data.iterrows():
    numeric_eliminator(row['Title'], index, 'Title')
# we print the time it took to preprocess whole titles
print((time.clock() - start_time)/60, "minutes")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


6.176678787111492 minutes


In [5]:
movies_genres_data.to_csv("movies_data.csv",index=False)

# Content Based Recommendation System

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
movies=pd.read_csv('movies.csv')

In [4]:
tf = TfidfVectorizer(analyzer='word',min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['Genres'])

In [5]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:
titles = movies['Title']
indices = pd.Series(movies.index, index=movies['Title'])

In [7]:
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [8]:
genre_recommendations('Good Will Hunting (1997)').head(20)

25                                        Othello (1995)
26                                   Now and Then (1995)
29     Shanghai Triad (Yao a yao yao dao waipo qiao) ...
30                                Dangerous Minds (1995)
35                               Dead Man Walking (1995)
39                       Cry, the Beloved Country (1995)
42                                    Restoration (1995)
52                                       Lamerica (1994)
54                                        Georgia (1995)
56                          Home for the Holidays (1995)
61                             Mr. Holland's Opus (1995)
66                                       Two Bits (1995)
77                            Crossing Guard, The (1995)
79          White Balloon, The (Badkonake Sefid ) (1995)
81                       Antonia's Line (Antonia) (1995)
82       Once Upon a Time... When We Were Colored (1995)
89                    Journey of August King, The (1995)
92                             

In [9]:
genre_recommendations('Heat (1995)').head(20)

168                      Hackers (1995)
2209                       Ronin (1998)
2887    Someone to Watch Over Me (1987)
3694                         F/X (1986)
3695                       F/X 2 (1992)
1884      French Connection, The (1971)
2471              Corruptor, The (1999)
46                 Seven (Se7en) (1995)
49           Usual Suspects, The (1995)
517            Romeo Is Bleeding (1993)
653                  Purple Noon (1960)
1073              Reservoir Dogs (1992)
1331            Albino Alligator (1996)
1444            City of Industry (1997)
1601                 Playing God (1997)
1629                   Incognito (1997)
1640                  Red Corner (1997)
2140          Young and Innocent (1937)
2322              Simple Plan, A (1998)
2492                  True Crime (1999)
Name: Title, dtype: object

# Memory based Collabarative System

In [2]:
ratingsdata=pd.read_csv('ratings.csv')

In [3]:
ratingsdata.groupby('UserID').count()

,MovieID,Rating,Timestamp
UserID,,,
1,53,53,53
2,129,129,129
3,51,51,51
4,21,21,21
5,198,198,198
6,71,71,71
7,31,31,31
8,139,139,139
9,106,106,106


In [47]:
# Fill NaN values in user_id and movie_id column with 0
ratingsdata['UserID'] = ratingsdata['UserID'].fillna(0)
ratingsdata['MovieID'] = ratingsdata['MovieID'].fillna(0)

# Replace NaN values in rating column with average of all values
ratingsdata['Rating'] = ratingsdata['Rating'].fillna(ratingsdata['Rating'].mean())

# Implementation of Memory Based Collabarative Filtering

In [48]:
np.random.seed(0)
ratingdata=ratingsdata.reindex(np.random.permutation(ratingsdata.index))
small_data = ratingsdata.sample(frac=0.01)

In [49]:
small_data.head()

,UserID,MovieID,Rating,Timestamp
659647,3975,21,3,965668304
961674,5795,2664,4,958149595
669195,4022,3706,3,965526445
936333,5649,1573,4,958867634
418980,2529,1033,3,974568481


In [50]:
small_data.groupby('UserID').count()

,MovieID,Rating,Timestamp
UserID,,,
3,1,1,1
5,4,4,4
8,1,1,1
9,2,2,2
10,7,7,7
12,1,1,1
13,1,1,1
17,1,1,1
18,3,3,3


In [51]:
small_data.groupby('MovieID').count()

,UserID,Rating,Timestamp
MovieID,,,
1,23,23,23
2,7,7,7
3,8,8,8
4,3,3,3
5,5,5,5
6,9,9,9
7,6,6,6
9,1,1,1
10,6,6,6


In [52]:
small_data.shape

(10002, 4)

In [53]:
small_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10002 entries, 659647 to 658593
Data columns (total 4 columns):
UserID       10002 non-null int64
MovieID      10002 non-null int64
Rating       10002 non-null int64
Timestamp    10002 non-null int64
dtypes: int64(4)
memory usage: 390.7 KB


In [54]:
small_data=small_data[['UserID','MovieID','Rating']]

In [55]:
from sklearn.model_selection import train_test_split,cross_val_score
train_data, test_data = train_test_split(small_data, test_size=0.2,random_state=42)

In [56]:
train_data.shape

(8001, 3)

In [57]:
test_data.shape

(2001, 3)

In [68]:
train_data_matrix = train_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])
train_data_matrix=train_data_matrix.astype(np.float64, copy=False)
test_data_matrix = test_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])
test_data_matrix=test_data_matrix.astype(np.float64, copy=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [69]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])

[[1.         0.95634215 0.99999043 0.91374367]
 [0.95634215 1.         0.95761137 0.99258992]
 [0.99999043 0.95761137 1.         0.91551214]
 [0.91374367 0.99258992 0.91551214 1.        ]]


In [70]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    mean_user_rating = ratings.mean(axis=1)
    # Use np.newaxis so that mean_user_rating has same format as ratings
    ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
    pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    return pred

In [71]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [72]:
user_prediction = predict(train_data_matrix, user_correlation, type='user')

In [73]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [66]:
type(train_data)

pandas.core.frame.DataFrame